# The Chat Format

In this notebook, you will explore how you can utilize the chat format to have extended conversations with chatbots personalized or specialized for specific tasks or behaviors.

In [1]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [2]:
client = openai.OpenAI()

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message.content
    
def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message.content

In [8]:
help(openai.ChatCompletion)

Help on APIRemovedInV1Proxy in module openai.lib._old_api:

APIRemovedInV1Proxy


In [7]:
help(client.chat.completions)

Help on Completions in module openai.resources.chat.completions.completions object:

class Completions(openai._resource.SyncAPIResource)
 |  Completions(client: 'OpenAI') -> 'None'
 |
 |  Method resolution order:
 |      Completions
 |      openai._resource.SyncAPIResource
 |      builtins.object
 |
 |  Methods defined here:
 |
 |  create(
 |      self,
 |      *,
 |      messages: 'Iterable[ChatCompletionMessageParam]',
 |      model: 'Union[str, ChatModel]',
 |      audio: 'Optional[ChatCompletionAudioParam] | NotGiven' = NOT_GIVEN,
 |      frequency_penalty: 'Optional[float] | NotGiven' = NOT_GIVEN,
 |      function_call: 'completion_create_params.FunctionCall | NotGiven' = NOT_GIVEN,
 |      functions: 'Iterable[completion_create_params.Function] | NotGiven' = NOT_GIVEN,
 |      logit_bias: 'Optional[Dict[str, int]] | NotGiven' = NOT_GIVEN,
 |      logprobs: 'Optional[bool] | NotGiven' = NOT_GIVEN,
 |      max_completion_tokens: 'Optional[int] | NotGiven' = NOT_GIVEN,
 |      max_t

In [15]:
messages =  [  
{'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},    #system or developer roles both work as desired
{'role':'user', 'content':'tell me a joke'},   
{'role':'assistant', 'content':'Why did the chicken cross the road'},   
{'role':'user', 'content':'I don\'t know'}  ]

In [16]:
response = get_completion_from_messages(messages, temperature=1)
print(response)

To get to the other side! Haha, a jest that never fails to amuse the masses.


## No memory without context
Stand alone interaction in each conversation

In [18]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Hi, my name is Isa'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Yes,  can you remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Hello Isa! It's nice to meet you. How can I assist you today?
I'm sorry, but I don't have the ability to remember personal information like your name. How can I assist you today?


In [20]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Isa'},
{'role':'assistant', 'content': "Hi Isa! It's nice to meet you. \
Is there anything I can help you with today?"},
{'role':'user', 'content':'Yes, you can remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Your name is Isa.


In [32]:
"hello \
world"

'hello world'

In [34]:
"hello " \
"world"

'hello world'

In [36]:
"""hello \
world"""

'hello world'

In [38]:
"""hello 
world"""

'hello \nworld'

# OrderBot
We can automate the collection of user prompts and assistant responses to build a  OrderBot. The OrderBot will take orders at a pizza restaurant. 

In [17]:
def collect_messages(_): #inp and panels are defined later
    prompt = inp.value
    inp.value = ''
    
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    
    panels.append(pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(pn.Row('Assistant:', pn.pane.Markdown(response, width=600, styles={'background-color': '#F6F6F6'})))

    return pn.Column(*panels)

In [4]:
import panel as pn # GUI
pn.extension() # Initialize Panel

In [5]:
context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ]  # accumulate messages

context[0]["content"]

"\nYou are OrderBot, an automated service to collect orders for a pizza restaurant. You first greet the customer, then collects the order, and then asks if it's a pickup or delivery. You wait to collect the entire order, then summarize it and check for a final time if the customer wants to add anything else. If it's a delivery, you ask for an address. Finally you collect the payment.Make sure to clarify all options, extras and sizes to uniquely identify the item from the menu.You respond in a short, very conversational friendly style. The menu includes pepperoni pizza  12.95, 10.00, 7.00 cheese pizza   10.95, 9.25, 6.50 eggplant pizza   11.95, 9.75, 6.75 fries 4.50, 3.50 greek salad 7.25 Toppings: extra cheese 2.00, mushrooms 1.50 sausage 3.00 canadian bacon 3.50 AI sauce 1.50 peppers 1.00 Drinks: coke 3.00, 2.00, 1.00 sprite 3.00, 2.00, 1.00 bottled water 5.00 "

In [22]:
panels = []
inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")
interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='Chat!')
    [2] ParamFunction(function, _pane=Column, defer_load=False, height=300, loading_indicator=True)

In [23]:
inp

TextInput(placeholder='Enter text here…')

In [35]:
pn.Column(*panels)

Column
    [0] Row
        [0] Markdown(str)
        [1] Markdown(str, width=600)
    [1] Row
        [0] Markdown(str)
        [1] Markdown(str, styles={'background-color': '...}, width=600)
    [2] Row
        [0] Markdown(str)
        [1] Markdown(str, width=600)
    [3] Row
        [0] Markdown(str)
        [1] Markdown(str, styles={'background-color': '...}, width=600)
    [4] Row
        [0] Markdown(str)
        [1] Markdown(str, width=600)
    [5] Row
        [0] Markdown(str)
        [1] Markdown(str, styles={'background-color': '...}, width=600)
    [6] Row
        [0] Markdown(str)
        [1] Markdown(str, width=600)
    [7] Row
        [0] Markdown(str)
        [1] Markdown(str, styles={'background-color': '...}, width=600)
    [8] Row
        [0] Markdown(str)
        [1] Markdown(str, width=600)
    [9] Row
        [0] Markdown(str)
        [1] Markdown(str, styles={'background-color': '...}, width=600)
    [10] Row
        [0] Markdown(str)
        [1] Markdown(str, width=600)
    [11] Row
        [0] Markdown(str)
        [1] Markdown(str, styles={'background-color': '...}, width=600)
    [12] Row
        [0] Markdown(str)
        [1] Markdown(str, width=600)
    [13] Row
        [0] Markdown(str)
        [1] Markdown(str, styles={'background-color': '...}, width=600)
    [14] Row
        [0] Markdown(str)
        [1] Markdown(str, width=600)
    [15] Row
        [0] Markdown(str)
        [1] Markdown(str, styles={'background-color': '...}, width=600)
    [16] Row
        [0] Markdown(str)
        [1] Markdown(str, width=600)
    [17] Row
        [0] Markdown(str)
        [1] Markdown(str, styles={'background-color': '...}, width=600)
    [18] Row
        [0] Markdown(str)
        [1] Markdown(str, width=600)
    [19] Row
        [0] Markdown(str)
        [1] Markdown(str, styles={'background-color': '...}, width=600)
    [20] Row
        [0] Markdown(str)
        [1] Markdown(str, width=600)
    [21] Row
        [0] Markdown(str)
        [1] Markdown(str, styles={'background-color': '...}, width=600)
    [22] Row
        [0] Markdown(str)
        [1] Markdown(str, width=600)
    [23] Row
        [0] Markdown(str)
        [1] Markdown(str, styles={'background-color': '...}, width=600)
    [24] Row
        [0] Markdown(str)
        [1] Markdown(str, width=600)
    [25] Row
        [0] Markdown(str)
        [1] Markdown(str, styles={'background-color': '...}, width=600)
    [26] Row
        [0] Markdown(str)
        [1] Markdown(str, width=600)
    [27] Row
        [0] Markdown(str)
        [1] Markdown(str, styles={'background-color': '...}, width=600)
    [28] Row
        [0] Markdown(str)
        [1] Markdown(str, width=600)
    [29] Row
        [0] Markdown(str)
        [1] Markdown(str, styles={'background-color': '...}, width=600)

In [28]:
panels

[Row
     [0] Markdown(str)
     [1] Markdown(str, width=600),
 Row
     [0] Markdown(str)
     [1] Markdown(str, styles={'background-color': '...}, width=600)]

In [40]:
context

[{'role': 'system',
  'content': "\nYou are OrderBot, an automated service to collect orders for a pizza restaurant. You first greet the customer, then collects the order, and then asks if it's a pickup or delivery. You wait to collect the entire order, then summarize it and check for a final time if the customer wants to add anything else. If it's a delivery, you ask for an address. Finally you collect the payment.Make sure to clarify all options, extras and sizes to uniquely identify the item from the menu.You respond in a short, very conversational friendly style. The menu includes pepperoni pizza  12.95, 10.00, 7.00 cheese pizza   10.95, 9.25, 6.50 eggplant pizza   11.95, 9.75, 6.75 fries 4.50, 3.50 greek salad 7.25 Toppings: extra cheese 2.00, mushrooms 1.50 sausage 3.00 canadian bacon 3.50 AI sauce 1.50 peppers 1.00 Drinks: coke 3.00, 2.00, 1.00 sprite 3.00, 2.00, 1.00 bottled water 5.00 "},
 {'role': 'user', 'content': 'Hi'},
 {'role': 'assistant',
  'content': 'Hello! Welcome t

In [41]:
type(context)

list

In [42]:
help(context.copy)

Help on built-in function copy:

copy() method of builtins.list instance
    Return a shallow copy of the list.



In [43]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'create a json summary of the previous food order. Itemize the price for each item\
 The fields should be 1) pizza, include size 2) list of toppings 3) list of drinks, include size   4) list of sides include size  5)total price '},    
)
 #The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},    

response = get_completion_from_messages(messages, temperature=0)
print(response)

```json
{
    "pizza": {
        "size": "Large Cheese Pizza",
        "toppings": ["Peppers", "Sausage"]
    },
    "drinks": [],
    "sides": [],
    "total_price": 16.70
}
```
